# Exercício com o PyAccel
Este é um exercício para treino do uso do PyAccel no projeto de uma linha de transporte.

## Definção:

A linha de transporte começa assim:
1) Espaço livre de 10 cm
2) Lente fina quadrupolar de foco f1
4) Lente fina quadrupolar de foco f2
3) Espaço livre de 20 cm
4) Eletroímã curvador de raio 0.5 m e ângulo de 30 graus
5) Espaço livre de 10 cm

## Tarefa: 

Criar uma linha de trasporte de 3 metros que produza um menor feixe possível!

In [ ]:
## Preparando o ambiente
if 'google.colab' in str(get_ipython()):
  !git clone https://github.com/tiagofiorini/pyaccel.git
  import pyaccel.pyaccel as pyaccel
else:
  import pyaccel as pyaccel

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

import importlib
importlib.reload(pyaccel)

%matplotlib inline

# Primeiro passo: Otimizar as lentes antes do dipolo

In [ ]:
@widgets.interact(theta=(0.0, 90.0, 1.0), f1=(0.1,10.0,0.005), f2=(0.1,10.0,0.005))
def fn0(theta=30.0,f1=10.0, f2=10.0):    
    TL = pyaccel.InitilazeTransportLine( sx = 2.0, sxp = 0.01, sy = 2.0, syp = 0.01, sz = 1.0, sdppp0 = 5e-3 , 
                                    m0 = pyaccel.Proton_mass_kg, e0 = pyaccel.Electron_charge, E0 = 2.0, num = 10000,
                                    XYdistribution='uniform')
    TL.AddDriftSpace (L=0.10)
    TL.AddThinLens(f=f1, setXdivergence=True, setYdivergence=False)
    TL.AddDriftSpace (L=0.10)    
    TL.AddThinLens(f=f2, setXdivergence=False, setYdivergence=True)
    TL.AddDriftSpace (L=0.20)
    TL.AddBendingMagnetX(BendingRadius = 0.5, BendingAngle = np.radians(30))
    TL.AddDriftSpace (L=0.10)
    TL.Propagate(N=7)
    TL.PlotBeamSpot(Rscale=1.0,MomentumColorScale=True)
    TL.PlotPhaseSpace(xlim=2.0,xplim=3.0,ylim=2.0,yplim=3.0)
    plt.figure()
    plt.subplot(121)
    plt.hist(TL.V[0,:])
    plt.subplot(122)
    plt.hist(TL.V[2,:])
    plt.show()    
    del TL

# Segundo passo: Otimizar uma linha de transporte para 3.0 m

In [ ]:
@widgets.interact(f1=(0.1,10.0,0.005), f2=(0.1,10.0,0.005), fA=(0.1, 10.0, 0.005), fB=(0.1, 10.0, 0.005), fC=(0.1, 10.0, 0.005))
def fn1(f1=10.0, f2=10.0, fA = 10.0, fB = 10.0, fC = 10.0, plot = True):
    TL = pyaccel.InitilazeTransportLine( sx = 2.0, sxp = 0.01, sy = 2.0, syp = 0.01, sz = 1.0, sdppp0 = 5e-3 , 
                                    m0 = pyaccel.Proton_mass_kg, e0 = pyaccel.Electron_charge, E0 = 2.0, num = 10000,
                                    XYdistribution='uniform')
    
    TL.AddDriftSpace (L=0.10)
    TL.AddThinLens(f=f1, setXdivergence=True, setYdivergence=False)
    TL.AddDriftSpace (L=0.10)    
    TL.AddThinLens(f=f2, setXdivergence=False, setYdivergence=True)
    TL.AddDriftSpace (L=0.20)
    TL.AddBendingMagnetX(BendingRadius = 0.5, BendingAngle = np.radians(30))
    TL.AddDriftSpace (L=0.10)

    ...
    
    TL.Propagate(N= ...)
    if plot == True:
        TL.PlotBeamSpot(Rscale=2.0,MomentumColorScale=True)
        TL.PlotPhaseSpace(xlim=2.0,xplim=3.0,ylim=2.0,yplim=3.0)
        plt.figure()
        plt.subplot(121)
        plt.hist(TL.V[0,:])
        plt.xlabel('X [mm]')
        plt.subplot(122)
        plt.hist(TL.V[2,:])
        plt.xlabel('Y [mm]')
        plt.show()
    r0 = np.sqrt( TL.V[0,:].std()**2 + TL.V[2,:].std()**2 )
    del TL
    return r0

# Desafio: Use uma função externa para uma otimização "fina" da configuração das lentes